# Knapsack example

- Maximize the value of the items
- Stay within the capacity of the knapsack
- Each item has a value, weight, and a volume
- The knapsack has both weight and volume capacity

In [1]:
values = [1, 2, 3, 4]
weights = [2, 2, 3, 3]
volumes = [3, 3, 2, 2]
n = 4
variables = list(range(n))
max_weight = 4
max_volume = 6

penalty_strength_weight = 20
penalty_strength_volume = 20

# Create the BQM model
Decision variable is binary variable $x$ for each item. If $x=1$, the item is in the knapsack, zero otherwise.

The coefficient of the variable is the negative of its value to account for the fact that we minimize the objective.

In [3]:
from dimod. import BinaryQuadraticModel

bqm = AdjVectorBQM('BINARY')

variables = [bqm.add_variable(f'x_{v}', -values[v]) for v in variables]

ModuleNotFoundError: No module named 'dimod.binary'

# Slack variables - Weight capacity

Inequality constraint:
$$\sum_i w_i x_i \leq W_c$$

We convert this type of constraint to an equality using a positive slack variable $S_w$ that accounts for the extra space.

$$\sum_i w_i x_i = W_c'$$
$$0 \leq W_c' \leq W_c$$
$$ S_w = W_c - W_c' > 0$$

The slack variable (an integer) can be represented using binary variables.

$$S_w = \sum_i^M y_i 2^i$$

The total number of binary variables used for this representation is the base 2 logarithm of the range of the slack variable.

$$M = \mathrm{log}_2 \left(W_c\right) + 1$$

In [44]:
import math
num_slacks_w = math.ceil(math.log2(max_weight)) + 1
slacks_weight = [bqm.add_variable(f's_w_{i}') for i in range(num_slacks_w)]

After adding the slack variables to the left-hand-side of the capacity constraint equation, we are left with an equality constraint that we can convert to a quadratic objective as before.


In [45]:
bqm.add_linear_equality_constraint(
    [(x, w) for x, w in zip(variables, weights)] + [(s_w, 2 ** i) for i, s_w in enumerate(slacks_weight)],
    constant=-max_weight,
    lagrange_multiplier=penalty_strength_weight
)

Similarly we define a set of slack variables for the volume capacity constraint.

In [46]:
num_slacks_v = math.ceil(math.log2(max_volume + 1))
slacks_volume = [bqm.add_variable(f's_v_{i}') for i in range(num_slacks_v)]

In [47]:
bqm.add_linear_equality_constraint(
    [(x, v) for x, v in zip(variables, volumes)] + [(s_v, 2 ** i) for i, s_v in enumerate(slacks_volume)],
    constant=-max_volume,
    lagrange_multiplier=penalty_strength_volume
)

For a small problem with less than 20 items, we can solve the QUBO exactly by enumerating all solutions.

In [48]:
from dimod import ExactSolver

response = ExactSolver().sample(bqm)
samples = response.samples()
best_solution = response.first.sample
print(response.truncate(5))

  s_v_0 s_v_1 s_v_2 s_w_0 s_w_1 s_w_2 x_0 x_1 x_2 x_3 energy num_oc.
0     0     0     1     1     0     0   0   0   0   1   -4.0       1
1     0     0     0     0     0     0   1   1   0   0   -3.0       1
2     0     0     1     1     0     0   0   0   1   0   -3.0       1
3     1     1     0     0     1     0   0   1   0   0   -2.0       1
4     1     1     0     0     1     0   1   0   0   0   -1.0       1
['BINARY', 5 rows, 5 samples, 10 variables]


Let's take a closer look at the solution.

- The main variables specify the knapsack assginment.
- The first set of slack variables show that the total weight of items is 1 unit less than the capacity
- The second set of slack variables show that the total volumeight of items is 1 unit less than the capacity

In [49]:
print({k: v for k, v in best_solution.items() if k in variables})
print({k: v for k, v in best_solution.items() if k in slacks_weight})
print({k: v for k, v in best_solution.items() if k in slacks_volume})

{'x_0': 0, 'x_1': 0, 'x_2': 0, 'x_3': 1}
{'s_w_0': 1, 's_w_1': 0, 's_w_2': 0}
{'s_v_0': 0, 's_v_1': 0, 's_v_2': 1}


In [50]:
def violations(sample, slack_variables, array, capacity):
    slack_total_weight = 0
    weight = 0
    for i, v in enumerate(slack_variables):
        slack_total_weight += sample[v] * 2 ** i
    for i, v in enumerate(variables):
        weight += array[i] * sample[v]
    print('Total of items: ', weight)
    print('Total slack value: ', slack_total_weight)
    print('Left hand side: ', weight + slack_total_weight)
    print('Satisfied? ', weight + slack_total_weight <= capacity)
    

In [51]:
print('Weight capacity constraint:')
violations(samples[0], slacks_weight, weights, max_weight)
print('Volume capacity constraint:')
violations(samples[0], slacks_volume, volumes, max_volume)

Weight capacity constraint:
Total of items:  3
Total slack value:  1
Left hand side:  4
Satisfied?  True
Volume capacity constraint:
Total of items:  2
Total slack value:  4
Left hand side:  6
Satisfied?  True
